In [ ]:
import cv2
import csv
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torchvision import transforms, models
from scipy.signal import correlate2d
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
import time
from psutil import virtual_memory
from skimage import io
from skimage.metrics import normalized_root_mse as ncc, structural_similarity as ssim
from scipy.ndimage.filters import gaussian_gradient_magnitude
from sklearn.base import BaseEstimator
from scipy.interpolate import griddata
from sklearn.cluster import KMeans
import os

In [ ]:
def normalize_image(image):
    min_val = image.min()
    max_val = image.max()
    if max_val != min_val:
        image = (image - min_val) / (max_val - min_val) * 65535
    return image.astype('uint16')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def likelihood(error, sigma):
    return np.exp(-error**2 / (2 * sigma**2))

class HomographyModel(BaseEstimator):
    def fit(self, X, y=None):
        src_pts = X[:, :2].reshape(-1, 1, 2)
        dst_pts = X[:, 2:].reshape(-1, 1, 2)
        self.H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)
        return self

    def errors(self, X, y=None):
        src_pts = X[:, :2].reshape(-1, 1, 2)
        dst_pts = X[:, 2:].reshape(-1, 1, 2)
        dst_pts_estimated = cv2.perspectiveTransform(src_pts, self.H)
        return np.sqrt(np.sum((dst_pts - dst_pts_estimated)**2, axis=2)).ravel()

def mlesac(data, model_class, min_samples, max_iterations, sigma, threshold):
    best_model = None
    best_likelihood = 0
    best_inliers = None

    for _ in range(max_iterations):
        sample = data[np.random.choice(data.shape[0], min_samples, replace=False)]
        model = model_class()
        model.fit(sample)
        errors = model.errors(data)
        inliers = data[errors <= threshold]
        outliers = data[errors > threshold]
        likelihood_inliers = np.sum(likelihood(errors[errors <= threshold], sigma))
        likelihood_outliers = np.sum(likelihood(errors[errors > threshold], sigma))
        likelihood_total = likelihood_inliers + likelihood_outliers
        if likelihood_total > best_likelihood:
            best_model = model
            best_likelihood = likelihood_total
            best_inliers = inliers

    return best_model, best_inliers

def draw_keypoint_matches(img1, img2, keypoints1, keypoints2, matches, adaptive_threshold, save_path):
    good_matches = [m for m in matches if m.distance <= adaptive_threshold]
    img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.imwrite(save_path, img_matches)

def adjust_brightness(img, alpha=0.0, beta=0.0):
    return cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

def adjust_contrast(img, alpha=0.0, beta=0.0):
    return cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

def denoise_image(img, weight=0.1):
    return img

def preprocess_image(img):
    img = adjust_brightness(img, alpha=1.0, beta=0.0)
    img = adjust_contrast(img, alpha=1.0, beta=0.0)
    img = denoise_image(img, weight=0.0)
    return img

def open_images(fixed_image_path, moving_image_paths):
    fixed_image = cv2.imread(fixed_image_path)
    moving_images = [cv2.imread(path) for path in moving_image_paths]
    fixed_image = preprocess_image(fixed_image)
    moving_images = [preprocess_image(img) for img in moving_images]

    return fixed_image, moving_images

def features_to_keypoints_and_descriptors(features_np):
    h, w = int(np.sqrt(features_np.size)), int(np.sqrt(features_np.size))
    img = cv2.resize(features_np, (h, w))
    img = (img * 255).astype(np.uint8)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)

    return keypoints, descriptors

def extract_combined_features(image):
    sift = cv2.SIFT_create()
    sift_keypoints, sift_descriptors = sift.detectAndCompute(image, None)
    return sift_keypoints, sift_descriptors

def homography_registration(img1, img2, method=cv2.RANSAC, threshold=5.0, threshold_multiplier=5.0):
    keypoints1, descriptors1 = extract_combined_features(img1)
    keypoints2, descriptors2 = extract_combined_features(img2)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    matches = sorted(good_matches, key=lambda x: x.distance)
    median_distance = np.median([m.distance for m in matches])
    adaptive_threshold = median_distance * threshold_multiplier

    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    data = np.hstack([src_pts.reshape(-1, 2), dst_pts.reshape(-1, 2)])
    model, inliers = mlesac(data, HomographyModel, min_samples=4, max_iterations=1000, sigma=1.0, threshold=adaptive_threshold)
    H = model.H

    print(f"Number of key points found: {len(matches)}")
    print(f"Adaptive threshold used: {adaptive_threshold}")
    return H, matches, adaptive_threshold, keypoints1, keypoints2

def draw_and_save_keypoints(image, keypoints, save_path, color=(218, 232, 28), thickness=3, radius=4):
    image_with_keypoints = image.copy()
    for kp in keypoints:
        center = (int(kp.pt[0]), int(kp.pt[1]))
        cv2.circle(image_with_keypoints, center, radius, color, thickness)
    cv2.imwrite(save_path, image_with_keypoints)

def keypoints_to_csv(keypoints, descriptors, file_name):
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Keypoint ID', 'X Coordinate', 'Y Coordinate', 'Scale', 'Orientation', 'Descriptor Vector'])
        for idx, kp in enumerate(keypoints):
            descriptor_str = ' '.join(map(str, descriptors[idx]))
            writer.writerow([idx, kp.pt[0], kp.pt[1], kp.size, kp.angle, descriptor_str])

def save_match_details(matches, keypoints1, keypoints2, file_name):
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Match ID', 'Keypoint ID (Fixed)', 'Keypoint ID (Moving)', 'Distance/Similarity Score',
                         'Displacement Vector Δx', 'Displacement Vector Δy', 'Scale Difference', 'Orientation Difference'])

        for idx, match in enumerate(matches):
            kp1 = keypoints1[match.queryIdx]
            kp2 = keypoints2[match.trainIdx]

            displacement_vector = (kp2.pt[0] - kp1.pt[0], kp2.pt[1] - kp1.pt[1])
            scale_difference = kp2.size - kp1.size
            orientation_difference = kp2.angle - kp1.angle

            writer.writerow([idx, match.queryIdx, match.trainIdx, match.distance,
                             displacement_vector[0], displacement_vector[1],
                             scale_difference, orientation_difference])

def save_outlier_details(matches, keypoints1, keypoints2, adaptive_threshold, file_name):
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Outlier ID', 'Keypoint ID (Fixed)', 'Keypoint ID (Moving)', 'Original Match ID'])

        outlier_id = 0
        for idx, match in enumerate(matches):
            if match.distance > adaptive_threshold:
                writer.writerow([outlier_id, match.queryIdx, match.trainIdx, idx])
                outlier_id += 1

if __name__ == '__main__':
    fixed_image_path = 'Fixed Image'
    moving_image_paths = ['Moving Image']
    fixed_image, moving_images = open_images(fixed_image_path, moving_image_paths)
    height, width = fixed_image.shape[:2]
    threshold_multiplier = 1

    for idx, img2 in enumerate(moving_images):
        start_time = time.time()
        H, matches, adaptive_threshold, keypoints1, keypoints2 = homography_registration(fixed_image, img2, threshold_multiplier=threshold_multiplier)
        result = cv2.warpPerspective(img2, H, (width, height), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        end_time = time.time()
        inliers = np.sum([1 for m in matches if m.distance <= adaptive_threshold])
        Registration_time = end_time - start_time
        print("Registration_time (s): ", Registration_time)
        print(f"Image {idx + 1}: Number of inliers with adaptive threshold: {inliers}")
        cv2.imwrite(f'Registered Image', result)
        fixed_threshold = 20
        inliers = np.sum([1 for m in matches if m.distance <= fixed_threshold])

        fixed_sift_keypoints, fixed_sift_descriptors = extract_combined_features(fixed_image)
        moving_sift_keypoints, moving_sift_descriptors = extract_combined_features(img2)

        print(f"Image {idx + 1}: Number of inliers without adaptive threshold: {inliers}")
